######  fMRI Decoding Project Showcase

 


###### fMRI Decoding showcase

In [2]:
from nilearn import datasets, image, plotting
from nilearn.input_data import NiftiMasker
from nilearn.image.image import mean_img
from nilearn.image import index_img
import pandas as pd


Preprocessing the Haxby Dataset


In [3]:
#import Haxby et al.(2001): Faces and Objects in Ventral Temporal Cortex (fMRI)
# Subjects 5 and 6 don't have complete label or anatomical information, only included subjects 1-4
haxby_dataset = datasets.fetch_haxby(subjects=4)

#load nifti images for the given subjects. Range 0-3
#defaults to subject 2
def loadSubject(subjectNum = 1):
    # 'func' is a list of filenames: one for each subject
    fmri_filename = haxby_dataset.func[subjectNum]
    # print basic information on the dataset
    print('First subject functional nifti images (4D) are at: %s' %
          fmri_filename)  # 4D data
    return fmri_filename

#plotting subject's anatomical brain
def plotAnat(subjectNum = 1):
    path = haxby_dataset.anat[subjectNum]
    plotting.plot_stat_map(path, threshold=3)
    plotting.show()

#plotting mean functionam MRI
def plotMeanFunc(subjectNum = 1):
    mean_haxby = mean_img(haxby_dataset.func[subjectNum])
    plotting.plot_stat_map(mean_haxby, threshold=3)
    plotting.show()

#plotting one random scan of fMRI
def plotRandomFunc(subjectNum = 1):
    rand_func = index_img(haxby_dataset.func[subjectNum], 30)
    plotting.plot_stat_map(rand_func, threshold=3)
    plotting.show()

fmri_filename = loadSubject(0)
plotAnat(subjectNum = 2)
plotMeanFunc(2)
plotRandomFunc(2)

behavioral = pd.read_csv(haxby_dataset.session_target[0], sep=" ")
conditions = behavioral['labels']

facecat_mask = conditions.isin(['face', 'cat'])
conditions_facecat = conditions[facecat_mask]
session_facecat = behavioral[facecat_mask].to_records(index = False)

facehouse_mask = conditions.isin(['face', 'house'])
conditions_facehouse = conditions[facehouse_mask]
session_facehouse = behavioral[facehouse_mask].to_records(index = False)

threeway_mask = conditions.isin(['face', 'house', 'cat'])
conditions_threeway = conditions[threeway_mask]
session_threeway = behavioral[threeway_mask].to_records(index = False)
print("Number of trials: ", len(conditions_threeway))
mask_filename = haxby_dataset.mask
#masking the data from 4D image to 2D array: voxel x time
#with smothing and standardization
masker = NiftiMasker(mask_img=mask_filename, smoothing_fwhm=4, standardize=True, memory="nilearn_cache", memory_level=1)
print(haxby_dataset.mask)
X = masker.fit_transform(fmri_filename)

# Apply our condition_mask
FC = X[facecat_mask]

FH = X[facehouse_mask]

FHC = X[threeway_mask]
#three-way classification with NN
FHC_train = FHC[:250]
conditions_train = conditions_threeway[1:250]
FHC_val = FHC[250:]
Y_val = conditions_threeway[250:]



First subject functional nifti images (4D) are at: /home/zc23/nilearn_data/haxby2001/subj1/bold.nii.gz


/home/zc23/CS344/venv/lib/python3.5/site-packages/matplotlib/figure.py:2369: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


Number of trials:  324
/home/zc23/nilearn_data/haxby2001/mask.nii.gz


![title](anatomical.png)

In [7]:
from sklearn.svm import SVC
from sklearn.feature_selection import SelectPercentile, f_classif, SelectKBest
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from nilearn import image
from nilearn.plotting import plot_stat_map, show
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

# Define the dimension reduction to be used.
# Here we use a classical univariate feature selection based on F-test,
# namely Anova. When doing full-brain analysis, it is better to use
# SelectPercentile, keeping 5% of voxels
# (because it is independent of the resolution of the data).
feature_selection = SelectPercentile(f_classif, percentile=5)

#one-vs-the-rest
#cited from https://scikit-learn.org/stable/modules/svm.html#multi-class-classification
lin_svc = LinearSVC()
facecathouse_svc = Pipeline([('anova', feature_selection), ('svc', lin_svc)])
facecathouse_svc.fit(FHC, conditions_threeway)
#
# another_svc = OneVsRestClassifier(Pipeline([('anova', SelectKBest(f_classif, k=500)), ('svc', SVC(kernel = 'linear'))]))
# another_svc.fit(FHC, conditions_threeway)

# Output accuracy
# Define the cross-validation scheme used for validation.
# Here we use a LeaveOneGroupOut cross-validation on the session group
# which corresponds to a leave-one-session-out
def modelAccuracy(model, X, conditions, groups):
    cv = LeaveOneGroupOut()

    # Compute the prediction accuracy for the different folds (i.e. session)
    cv_scores = cross_val_score(model, X, conditions, cv=cv, groups=groups)

    # Return the corresponding mean prediction accuracy
    classification_accuracy = cv_scores.mean()

    # Print the results
    print("Classification accuracy: %.4f / Chance level: %f" %
          (classification_accuracy, 1. / len(conditions.unique())))

print("Support Vector Model with linear kernel cross validation score accuracy: ")
modelAccuracy(facecathouse_svc, FHC, conditions_threeway, session_threeway)

# print("The second model on face vs cat vs house: ")
# modelAccuracy(another_svc, FHC, conditions_threeway, session_threeway)

cross_validation = cross_val_score(facecathouse_svc, FHC, conditions_threeway, cv = 4, verbose = 1)
print("Support Vector Model with linear kernel cross validation score: ", cross_validation.mean())



Support Vector Model with linear kernel cross validation score accuracy: 


Classification accuracy: 0.6265 / Chance level: 0.333333


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Support Vector Model with linear kernel cross validation score:  0.7993827160493827


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.8s finished


Using an NN model

In [6]:
import numpy as np
from keras import models
from keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder, StandardScaler

#need to one-hot encode the Y labels
enc = OneHotEncoder()
#cited from https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
Y = enc.fit_transform(conditions_threeway[:, np.newaxis]).toarray()
labels_train = Y[:250]
labels_val = Y[250:]


# experimental attempt to transform the 2D voxel*time array
reshaped_X = np.empty(shape = (324, 798))
reshaped_dimension = np.empty(shape = (1, 798))
subject = []
dim = []
# for dimension in range(0, 324):
#     for x in range(0, 797):
#         sum_series = 0
#         for i in range(0, 49):
#             index = x * 50 + i
#             sum_series += FHC[dimension][index]
#         dim.append(sum_series/49)
#     subject.append(dim)
#
# reshaped_X = np.array(subject)

for i in range(0, 324):
    old_dim = FHC[i]
    new_dim = np.mean(old_dim[:(len(old_dim)// 50) * 50].reshape(-1, 50), axis=1)
    subject.append(new_dim)

reshaped_X = np.array(subject)
print(reshaped_X.shape)
print(reshaped_X)

reshaped_Xtrain = reshaped_X[:250]
reshaped_Xval = reshaped_X[250:]

# model = models.Sequential()
# # model.add(Dense(64, input_dim = 39912, activation='relu'))
# # model.add(Dense(32, input_dim = 64, activation='relu'))
# model.add(Dense(16, input_dim = 39912, activation='relu'))
# # model.add(Dense(8, input_dim=16, activation='relu'))
# model.add(Dense(3, activation='softmax'))
# model.summary()
#
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.fit(FHC_train, labels_train, batch_size=5, epochs=50, verbose=1)
# score = model.evaluate(FHC_val, labels_val)
#
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

smaller_model = models.Sequential()
# smaller_model.add(Dense(64, input_dim = 798, activation='relu'))
# smaller_model.add(Dense(32, input_dim = 64, activation='relu'))
smaller_model.add(Dense(16, input_dim = 798, activation='relu'))
smaller_model.add(Dense(8, input_dim=16, activation='relu'))
smaller_model.add(Dense(3, activation='softmax'))
smaller_model.summary()

smaller_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
smaller_model.fit(reshaped_Xtrain, labels_train, batch_size=20, epochs=50, verbose=1)
score = smaller_model.evaluate(reshaped_Xval, labels_val)

print('Model with reduced input: ')
print('Training loss:', score[0])
print('Training accuracy:', score[1])


Using TensorFlow backend.


(324, 798)
[[-1.0773472  -1.0813442  -1.0147128  ...  0.54400927  0.9431207
   0.9707262 ]
 [-1.0518463  -1.0673845  -0.9953967  ...  0.55937266  0.9418709
   0.947163  ]
 [-1.0693469  -1.0831146  -1.0151976  ...  0.5424831   0.9371901
   0.9774819 ]
 ...
 [-0.96564937 -0.94586444 -0.92837715 ...  1.0298333   1.0321219
   1.069777  ]
 [-0.95019805 -0.92931074 -0.920926   ...  0.8451126   1.155889
   1.1264656 ]
 [-0.95977736 -0.93374336 -0.9239927  ...  0.9218399   1.2285094
   1.1368257 ]]
Instructions for updating:
Colocations handled automatically by placer.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                12784     
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 27        
Total params: 12,947
Trainable params: 12,947
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.


Epoch 1/50
 20/250 [=>............................] - ETA: 1s - loss: 1.1130 - acc: 0.3000

250/250 [==============================] - 0s 622us/step - loss: 1.0795 - acc: 0.3720


...

Epoch 50/50
 20/250 [=>............................] - ETA: 0s - loss: 0.0229 - acc: 1.0000

250/250 [==============================] - 0s 51us/step - loss: 0.0140 - acc: 1.0000


32/74 [===========>..................] - ETA: 0s

74/74 [==============================] - 0s 319us/step


Model with reduced input: 
Test loss: 2.2006091620471024
Test accuracy: 0.3648648664758012
